# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to benuklove@gmail.com and will expire on December 18, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Ben\AppData\Local\Temp\graphlab_server_1488423473.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000002  | 4.879058     | 6847177.773672     | 380824.455891 |

| 10        | 11       | 0.000002  | 5.307771     | 6842123.232651     | 364204.576180 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
all_features_weights = model_all.get("coefficients")
print all_features_weights

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
+------------------+-------+---------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [11]:
model_all["coefficients"].print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

In [30]:
# Find non-zero value total
investigate = model_all['coefficients']['value'].nnz()
print investigate

6


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

### bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above, and the intercept all have non-zero weights.  These are the features chosen by Lasso.

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [12]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [14]:
import numpy as np
from operator import sub

In [15]:
L1_dict = {}
for i in np.arange(1, 7.5, 0.5):
    L1_penalty = 10 ** i
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=L1_penalty, verbose=False)
    validation_results = model.predict(validation)
    diff = map(sub, validation_results, validation['price'])
    RSS = sum(map(lambda x: x ** 2, diff))
    L1_dict[L1_penalty] = RSS

In [16]:
L1_dict

{10.0: 625766285142461.4,
 31.622776601683793: 625766285362395.4,
 100.0: 625766286057886.9,
 316.2277660168379: 625766288257224.9,
 1000.0: 625766295212185.9,
 3162.2776601683795: 625766317206077.8,
 10000.0: 625766386760661.5,
 31622.776601683792: 625766606749281.4,
 100000.0: 625767302791633.2,
 316227.76601683791: 625769507643885.0,
 1000000.0: 625776517727025.8,
 3162277.6601683791: 625799062845466.9,
 10000000.0: 625883719085424.5}

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [17]:
best_L1 = min(L1_dict, key=L1_dict.get)
print "Best L1_penalty:", best_L1, "Minimum Validation Error:", L1_dict[min(L1_dict, key=L1_dict.get)]

Best L1_penalty: 10.0 Minimum Validation Error: 6.25766285142e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [22]:
selected_L1_model = graphlab.linear_regression.create(testing, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10, verbose=False)

In [23]:
selected_L1_model["coefficients"].print_rows(num_rows=18)

+------------------+-------+----------------+--------+
|       name       | index |     value      | stderr |
+------------------+-------+----------------+--------+
|   (intercept)    |  None | 15938.3491302  |  None  |
|     bedrooms     |  None | 7927.96151068  |  None  |
| bedrooms_square  |  None |  2811.6551531  |  None  |
|    bathrooms     |  None |  25090.812508  |  None  |
|   sqft_living    |  None | 39.6894164796  |  None  |
| sqft_living_sqrt |  None | 1115.10438309  |  None  |
|     sqft_lot     |  None | -0.15538020793 |  None  |
|  sqft_lot_sqrt   |  None | 126.735040031  |  None  |
|      floors      |  None | 19806.9697903  |  None  |
|  floors_square   |  None | 12646.6371612  |  None  |
|    waterfront    |  None | 413704.286579  |  None  |
|       view       |  None | 86807.2319587  |  None  |
|    condition     |  None | 5914.49546888  |  None  |
|      grade       |  None | 6017.72238936  |  None  |
|    sqft_above    |  None | 44.5451590346  |  None  |
|  sqft_ba

Hmmm, all of them?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [24]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [33]:
l1_penalty_values = np.logspace(8, 10, num=20)
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [36]:
L1_nonzero_dict = {}
for i in range(len(l1_penalty_values)):
    l1_pen = l1_penalty_values[i]
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pen, verbose=False)
    L1_nonzero_dict[l1_pen] = model['coefficients']['value'].nnz()

In [37]:
L1_nonzero_dict

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.11147901: 18,
 263665089.87303555: 17,
 335981828.62837881: 17,
 428133239.8719396: 17,
 545559478.11685145: 17,
 695192796.17755914: 17,
 885866790.41008317: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313128: 10,
 3792690190.7322536: 6,
 4832930238.5717525: 5,
 6158482110.6602545: 3,
 7847599703.5146227: 1,
 10000000000.0: 1}

For the following code, see below for desired terms 'L1_penalty_min and max'

In [54]:
# sort the dictionary by number of non-zero features selected by lasso regression model
from collections import OrderedDict
L1_nonzero_sorted = OrderedDict(sorted(L1_nonzero_dict.items(), key=lambda t: t[1]))
print L1_nonzero_sorted

OrderedDict([(10000000000.0, 1), (7847599703.5146227, 1), (6158482110.6602545, 3), (4832930238.5717525, 5), (3792690190.7322536, 6), (2976351441.6313128, 10), (2335721469.0901213, 12), (1832980710.8324375, 13), (1128837891.6846883, 15), (1438449888.2876658, 15), (885866790.41008317, 16), (545559478.11685145, 17), (263665089.87303555, 17), (335981828.62837881, 17), (695192796.17755914, 17), (428133239.8719396, 17), (100000000.0, 18), (206913808.11147901, 18), (127427498.57031322, 18), (162377673.91887242, 18)])


In [56]:
# find L1_penalty_min
for kv in L1_nonzero_sorted:
    if L1_nonzero_sorted[kv] > max_nonzeros:
        l1_penalty_min = kv
        break
print l1_penalty_min

# find L1_penalty_max
for kv in L1_nonzero_dict:
    if L1_nonzero_dict[kv] < max_nonzeros:
        l1_penalty_max = kv
        break
print l1_penalty_max

2976351441.63
3792690190.73


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [57]:
print "L1 penalty min is:", l1_penalty_min
print "L1 penalty max is:", l1_penalty_max

L1 penalty min is: 2976351441.63
L1 penalty max is: 3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [58]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [77]:
L1_sparse = []
for i in range(len(l1_penalty_values)):
    l1_pen = l1_penalty_values[i]
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pen, verbose=False)
    validation_results = model.predict(validation)
    diff = map(sub, validation_results, validation['price'])
    RSS = sum(map(lambda x: x ** 2, diff))
    L1_sparse.append((l1_pen, RSS, model['coefficients']['value'].nnz()))

In [78]:
L1_sparse

[(2976351441.6313128, 966925692362086.5, 10),
 (3019316638.952415, 974019450084557.2, 10),
 (3062281836.2735171, 981188367942451.0, 10),
 (3105247033.5946193, 989328342459472.0, 10),
 (3148212230.9157214, 998783211265885.9, 10),
 (3191177428.2368236, 1008477167020096.5, 10),
 (3234142625.5579257, 1018298780553818.6, 10),
 (3277107822.8790278, 1028247992205976.1, 10),
 (3320073020.20013, 1034616909232829.2, 8),
 (3363038217.5212321, 1038554735941039.9, 8),
 (3406003414.8423343, 1043237237871704.1, 8),
 (3448968612.1634364, 1046937488751713.4, 7),
 (3491933809.4845386, 1051147625612863.0, 7),
 (3534899006.8056407, 1055992735342999.4, 7),
 (3577864204.1267428, 1060799531763290.2, 7),
 (3620829401.447845, 1065707689498229.6, 6),
 (3663794598.7689471, 1069464335425584.1, 6),
 (3706759796.0900493, 1073504549585594.6, 6),
 (3749724993.4111514, 1077632775581412.0, 6),
 (3792690190.7322536, 1081867592324111.9, 6)]

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [85]:
# 1.
restricted = []
for x in L1_sparse:
    for y in x:
        if y == 7:
            restricted.append(x)
print "The L1 with lowest RSS on validation data with required sparsity is:", min(restricted)[0]

The L1 with lowest RSS on validation data with required sparsity is: 3448968612.16


In [99]:
# 2.
restricted_model = graphlab.linear_regression.create(validation, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3448968612.16, max_iterations=10, verbose=True)
print restricted_model["coefficients"].print_rows(num_rows=18)

Linear regression:

--------------------------------------------------------

Number of examples          : 9635

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 1.096428     | 6224466.704729     | 347180.749035 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 225253.486731 |  None  |
|     bedrooms     |  None | 675.957744394 |  None  |
| bedrooms_square  |  None | 14.9063764906 |  None  |
|    bathrooms     |  None |  14452.935551 |  None  |
|   sqft_living    |  None | 29.5010719299 |  None  |
| sqft_living_sqrt |  None | 659.218084958 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2941.12601894 |  None  |
|    sqft_above    |  None | 26.2948595039 |  None  |
|  sqft_basement   |  None |

See non-zero coefficients above for quiz question.